In [1]:
import ants
from petpal.kinetic_modeling.parametric_images import apply_rtm2_to_all_voxels, ReferenceTissueParametricImage
from petpal.utils.time_activity_curve import TimeActivityCurveFromFile
from petpal.preproc.image_operations_4d import brain_mask
import numpy as np

In [2]:
tac_path = '/export/scratch1/PETPAL/03Dec24/sub-PIB07018_ses-VYr12/tacs/WMRef_tac.tsv'
wmref = TimeActivityCurveFromFile(tac_path=tac_path)

In [3]:
pet_file = '/export/scratch1/PETPAL/03Dec24/sub-PIB07018_ses-VYr12/sub-PIB07018_ses-VYr12_moco.nii.gz'
pet_image = ants.image_read(filename=pet_file)
pet_array = pet_image.numpy()
mask_file = '/tmp/pet_mask.nii.gz'
mask_image = ants.image_read(mask_file)
mask_array = mask_image.numpy()

brain_mask(
    input_image_4d_path=pet_file,
    out_image_path=mask_file,
    atlas_image_path='/data/petsun43/data1/atlas/MNI152/MNI152_T1_2mm.nii',
    atlas_mask_path='/data/petsun43/data1/atlas/MNI152/MNI152_T1_2mm_brain_mask.nii.gz'
)

In [14]:
srtm2_array = apply_rtm2_to_all_voxels(
    tac_times_in_minutes=wmref.tac_times_in_minutes,
    tgt_image=pet_array,
    ref_tac_vals=wmref.tac_vals,
    mask_img=mask_array,
    method='srtm2',
    k2_prime=0.017
)

In [15]:
srtm2_image = ants.from_numpy(
    data=srtm2_array,
    origin=pet_image.origin,
    spacing=pet_image.spacing,
    direction=pet_image.direction
)

In [16]:
srtm2_image

ANTsImage
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (128, 128, 63, 2)
	 Spacing    : (2.0033, 2.0033, 2.425, 1.0)
	 Origin     : (127.2104, 127.2104, -75.175, 0.0)
	 Direction  : [-1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  1.]

In [17]:
ants.image_write(image=srtm2_image,filename='/tmp/srtm2_example.nii.gz')

In [4]:
frtm_bounds=[[0.5,0,10],[0.5,0,1],[0.5,0,1]]
srtm_bounds=[[0.5,0,1],[0.5,0,1]]

In [5]:
frtm2_array = apply_rtm2_to_all_voxels(
    tac_times_in_minutes=wmref.tac_times_in_minutes,
    tgt_image=pet_array,
    ref_tac_vals=wmref.tac_vals,
    mask_img=mask_array,
    method='frtm2',
    r1_bounds=[0.5,0,10],
    k3_bounds=[0.5,0,1],
    k4_bounds=[0.5,0,1],
    k2_prime=0.017
)

In [6]:
frtm2_image = ants.from_numpy(
    data=frtm2_array,
    origin=pet_image.origin,
    spacing=pet_image.spacing,
    direction=pet_image.direction
)
ants.image_write(image=frtm2_image,filename='/tmp/frtm2_example.nii.gz')

In [6]:
srtm2 = ReferenceTissueParametricImage(
    reference_tac_path=tac_path,
    pet_image_path=pet_file,
    mask_image_path=mask_file,
    output_directory='/tmp/rtm_parametric',
    output_filename_prefix='srtm2_class',
    method='srtm2'
)

In [13]:
srtm2_out = srtm2(k2_prime=0.017,image_scale=1)

In [12]:
srtm2.save_parametric_images()

In [5]:
frtm2 = ReferenceTissueParametricImage(
    reference_tac_path=tac_path,
    pet_image_path=pet_file,
    mask_image_path=mask_file,
    output_directory='/tmp/rtm_parametric',
    output_filename_prefix='frtm2_class',
    method='frtm2'
)

In [6]:
frtm2_out = frtm2(bounds=frtm_bounds,k2_prime=0.017,image_scale=1)